# <center> <img src="../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Lab 12** </center>

---

**Fecha**: 11 Mayo 2025

**Nombre del Equipo**: Arriba Linux

**Integrantes del Equipo**: Tirzah Peniche Barba / Ana Cristina Luna Arellano / Juan Pedro Bihouet

**Profesor**: Dr. Pablo Camarillo Ramirez

In [11]:
import findspark
findspark.init()

#### Creacion de la conexión con el cluster de spark


In [12]:
from pyspark.sql import SparkSession


# Create Spark session
spark = SparkSession.builder \
    .appName("ALS-Movie-Recommendation") \
    .master("spark://28d4ad191d34:7077") \
    .config("spark.ui.port","4040") \
    .getOrCreate()
sc = spark.sparkContext



In [13]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import col
from ArribaLinux.spark_utils import SparkUtils
from pyspark.sql.types import StructType, StructField, IntegerType, FloatType, LongType

#path
ratings_path = "/home/jovyan/notebooks/data/ratings.txt" 
#creamos el df con las columnas que nos pide
ratings_schema = StructType([
    StructField("userId", IntegerType(), True),
    StructField("movieId", IntegerType(), True),
    StructField("rating", FloatType(), True),
    StructField("timestamp", LongType(), True)
])

ratings_df = spark.read.csv(
    ratings_path,
    sep="::",              # <-- delimiter (el file no se me descargaba como .csv encontre esto para separarlo)
    schema=ratings_schema,
)


In [14]:
from pyspark.ml.recommendation import ALS
#generamos el modelo ALS
als = ALS(
    maxIter=10,
    regParam=0.1,
    userCol="userId",
    itemCol="movieId",
    ratingCol="rating",
    coldStartStrategy="drop"
)

In [15]:
model = als.fit(ratings_df)

In [16]:
predictions = model.transform(ratings_df)

predictions.show(truncate=False)

recommended = model.recommendForAllUsers(numItems=2)
recommended.show(truncate=False)

+------+-------+------+----------+----------+
|userId|movieId|rating|timestamp |prediction|
+------+-------+------+----------+----------+
|28    |0      |3.0   |1424380312|2.6242466 |
|28    |1      |1.0   |1424380312|1.1713365 |
|28    |2      |4.0   |1424380312|3.649982  |
|28    |3      |1.0   |1424380312|0.8008682 |
|28    |6      |1.0   |1424380312|0.9787566 |
|28    |7      |1.0   |1424380312|1.3701822 |
|28    |12     |5.0   |1424380312|3.5106254 |
|28    |13     |2.0   |1424380312|2.022234  |
|28    |14     |1.0   |1424380312|1.0674466 |
|28    |15     |1.0   |1424380312|1.0844364 |
|28    |17     |1.0   |1424380312|0.94172955|
|28    |19     |3.0   |1424380312|2.665203  |
|28    |20     |1.0   |1424380312|0.92025286|
|28    |23     |3.0   |1424380312|2.7755823 |
|28    |24     |3.0   |1424380312|2.2836795 |
|28    |27     |1.0   |1424380312|1.0695844 |
|28    |29     |1.0   |1424380312|1.1772664 |
|28    |33     |1.0   |1424380312|1.2561624 |
|28    |34     |1.0   |1424380312|

+------+----------------------------------+
|userId|recommendations                   |
+------+----------------------------------+
|20    |[{22, 3.6024003}, {94, 3.2383304}]|
|10    |[{92, 3.2280664}, {2, 3.121109}]  |
|0     |[{92, 3.0114312}, {2, 2.6349454}] |
|1     |[{68, 3.0046268}, {62, 2.9320545}]|
|21    |[{29, 4.3229423}, {52, 4.2465305}]|
|11    |[{30, 4.767297}, {32, 4.7413015}] |
|12    |[{46, 5.2744637}, {90, 4.6326585}]|
|22    |[{75, 4.5294313}, {51, 4.349877}] |
|2     |[{93, 4.490622}, {83, 4.335734}]  |
|13    |[{93, 2.979865}, {74, 2.8578439}] |
|3     |[{51, 4.097765}, {75, 4.0294666}] |
|23    |[{46, 4.794603}, {32, 4.746082}]  |
|4     |[{53, 3.4589245}, {29, 3.3570633}]|
|24    |[{52, 4.433993}, {69, 4.417164}]  |
|14    |[{52, 4.6393085}, {29, 4.550998}] |
|5     |[{55, 3.6768343}, {90, 3.5640485}]|
|15    |[{46, 3.9592383}, {90, 3.07912}]  |
|25    |[{71, 2.8628864}, {47, 2.752271}] |
|26    |[{51, 4.951104}, {22, 4.8334303}] |
|6     |[{25, 3.6855323}, {29, 3

In [ ]:
#Evaluamos el performance 
valuator = RegressionEvaluator(
    metricName="rmse",
    labelCol="rating",
    predictionCol="prediction"
)

rmse = evaluator.evaluate(predictions)
print(f"Root-mean-square error (RMSE): {rmse:.4f}")

Root-mean-square error (RMSE): 0.4010


In [8]:
sc.stop()